In [109]:
QUERY_TRANSLATE = [
    {
        "table_name": "colors",
        "table_type": "array",
        "query_command": ["color", "c"]
    },

    {
        "table_name": "color_identity",
        "table_type": "array",
        "query_command": ["id", "identity"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'type_line', 'type': 'string'}],
        "query_command": ["t", "type"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': '"set"', 'type': 'string'}, {'name':'set_name', 'type': 'string'}],
        "query_command": ["e", "edition", "s", "set"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'rarity', 'type': 'string'}],
        "query_command": ["r", "rarity"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'cmc', 'type': 'float'}],
        "query_command": ["cmc", "mv", "manavalue"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'oracle_text', 'type': 'string'}],
        "query_command": ["o", "oracle"]
    },

    {
        "table_name": "prices",
        "table_type": "object",
        "columns": [{'name': 'usd', 'type': 'float'}, {'name':'eur', 'type': 'float'}, {'name':'tix', 'type': 'float'}],
        "query_command": ["eur", "usd", "tix"]
    }
]

In [26]:
def decide_operator(query_string):
    operator = ""
    if ">=" in query_string:
        operator = ">="
    elif "<=" in query_string:
        operator = "<="
    elif ">" in query_string:
        operator = ">"
    elif "<" in query_string:
        operator = "<"
    elif ":" in query_string:
        operator = ":"

    return operator  

    # #Fajny approach, ale nie działał :(

    # gt_operator = query_string.find(">")
    # ge_operator = query_string.find(">=")
    # lt_operator = query_string.find("<")
    # le_operator = query_string.find("<=")
    # colon_operator = query_string.find(":")

    # operators = [">=", "<=", ">", "<", ":"]
    # found_operators = [gt_operator, ge_operator, lt_operator, le_operator, colon_operator]
    # return operators[found_operators.index(max(found_operators))]
    
    

In [91]:
def handle_query_when_number(table, operator, column, value, value_type):
    if value_type == "float":
        correct_value = float(value)
    elif value_type == "int":
        correct_value = int(value)
    
    correct_operator = operator if operator != ":" else "="
    query = f"{table}.{column} {operator} {str(correct_value)} AND "
    return query


In [126]:
def handle_query_when_string(table, column, value):
    value = value.replace("'","")
    query = f"{table}.{column} LIKE '%{value}%' AND "
    return query

In [18]:
def construct_query_join(table_name):
    join_query = f"""
    INNER JOIN {table_name}_table as {table_name}
    ON main.id = {table_name}.card_id
    """
    return join_query

In [133]:
def construct_query_when(query_string):
    where_query = "WHERE "
    query_elements = query_string.split("  ")
    for element in query_elements:
        operator = decide_operator(element)
        element_array = element.split(operator)

        argument = element_array[0]
        value = element_array[1]
        
        for translation in QUERY_TRANSLATE:
            translation_commands = translation["query_command"]
            translation_commands_negative = [f"-{element}" for element in translation_commands]
            if argument in translation_commands or argument in translation_commands_negative:
                if translation["table_type"] == "main":
                    if len(translation["columns"]) == 1:
                        column = translation["columns"][0]
                        column_name = column["name"]
                        column_type = column["type"]
                        
                        if column_type in ['float', 'int']:
                            query = handle_query_when_number("main", operator, column_name, value, column_type)
                            where_query += query
                        else:
                            query = handle_query_when_string("main", column_name, value)
                            where_query += query
                            
                elif translation["table_type"] == "array":
                    values = value.split(",")
                    query = ""
                    for value in values:
                        query += f"{translation['table_name']}.array_value {'LIKE' if argument in translation_commands else 'NOT LIKE'} '%{value}%' AND "
                    where_query += query

                elif translation["table_type"] == "object":
                    pass

    where_query = where_query[:-4]            
    return where_query

In [138]:
query_string = "o:'double strike'  cmc>3.0  cmc<6.0  color:r,g,b"
construct_query_when(query_string)

"WHERE main.oracle_text LIKE '%double strike%' AND main.cmc > 3.0 AND main.cmc < 6.0 AND colors.array_value LIKE '%r%' AND colors.array_value LIKE '%g%' AND colors.array_value LIKE '%b%' "

In [47]:
query_string = 'color:r,g -color:w'
def construct_query(query_string=None):
    query = 'SELECT id FROM main_table'
    query_commands = list(set([element.split(":")[0] for element in query_string.split(" ")]))

    tables_to_include = []
    for element in query_commands:
        for record in QUERY_TRANSLATE:
            if element in record['query_command']:
                tables_to_include.append(record['table_name'])

    for table in tables_to_include:
        join_query = construct_query_join(table)
        query += join_query
    
    return query

In [24]:

construct_query(query_string)

'SELECT id FROM main_table\n    INNER JOIN prices_table as prices\n    ON main.id = prices.card_id\n    \n    INNER JOIN colors_table as colors\n    ON main.id = colors.card_id\n    '